In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.0 MB/s eta 0:00:00


In [4]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import GPT2Tokenizer, GPT2Model
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to GPT-2 embeddings and return the dataset
def convert_to_gpt2_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Initialize GPT-2 model and tokenizer
    model = GPT2Model.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    # Add a padding token to the tokenizer
    tokenizer.pad_token = tokenizer.eos_token # or '[PAD]'


    # Function to get GPT-2 embeddings for a text
    def get_gpt2_embeddings(text):
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs).last_hidden_state
        return outputs.mean(dim=1).squeeze().numpy()

    # Generate embeddings for the "ANSWER" column
    embeddings = df[text_column].apply(lambda x: get_gpt2_embeddings(str(x)))

    # Stack embeddings into a 2D array
    embeddings = np.stack(embeddings.values)

    # Print the dimensions of the embeddings
    print(f"GPT-2 embeddings dimensions for '{text_column}': {embeddings.shape}")

    # Return the embeddings and the original DataFrame
    return embeddings, df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for GPT-2 embeddings
text_column = 'answer'

# Convert text column to GPT-2 embeddings and get the original DataFrame
embeddings, original_data = convert_to_gpt2_embeddings(input_file, text_column)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPT-2 embeddings dimensions for 'answer': (681, 768)
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.7101
Train Precision: 0.6932
Train Recall: 0.7101
Train F1 Score: 0.6949
Train Confusion Matrix:
[[ 11  11  10]
 [  4  56  72]
 [  2  39 271]]

Test Accuracy: 0.6488
Test Precision: 0.6247
Test Recall: 0.6488
Test F1 Score: 0.6365
Test Confusion Matrix:
[[  0   5   8]
 [  5  10  26]
 [  1  27 123]]

Running Random Forest model...
Random Forest Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7659
Test Precision: 0.7032
Test Recall: 0.7659
Test F1 Score: 0.7065
Test Confusion Matrix:
[[  0   1  12]
 [  0  10  31]
 [  0   4 147]]

Running Decision Tree model...
Decision Tree Model Metrics:
Train Accuracy: 0.99

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7024
Test Precision: 0.7279
Test Recall: 0.7024
Test F1 Score: 0.7129
Test Confusion Matrix:
[[  3   5   5]
 [  7  20  14]
 [  2  28 121]]

Running XGBoost model...
XGBoost Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.6964
Test Recall: 0.7268
Test F1 Score: 0.7062
Test Confusion Matrix:
[[  2   4   7]
 [  3  13  25]
 [  0  17 134]]

Running AdaBoost model...
AdaBoost Model Metrics:
Train Accuracy: 0.7941
Train Precision: 0.7919
Train Recall: 0.7941
Train F1 Score: 0.7873
Train Confusion Matrix:
[[ 19  13   0]
 [  0  74  58]
 [  0  27 285]]

Test Accuracy: 0.6976
Test Precision: 0.6910
Test Recall: 0.6976
Test F1 Score: 0.6

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model Metrics:
Train Accuracy: 0.8739
Train Precision: 0.8716
Train Recall: 0.8739
Train F1 Score: 0.8719
Train Confusion Matrix:
[[ 23   6   3]
 [  2 100  30]
 [  3  16 293]]

Test Accuracy: 0.7317
Test Precision: 0.7344
Test Recall: 0.7317
Test F1 Score: 0.7298
Test Confusion Matrix:
[[  1   8   4]
 [  1  24  16]
 [  4  22 125]]

Running SVM model...
SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]

